In [6]:
import pandas as pd
from datetime import datetime
from datetime import date, timedelta

In [23]:
ZSDKAP_FILE_PATH = 'excel_files/zsdkap2.xlsx'
ZSBE_FILE_PATH = 'excel_files/ZSBE_L1K.xlsx'
MB5T_FROM_2101_TO_ALL_PLANTS_FILE_PATH = 'excel_files/MB5T_from_2101_to_all_plants.xlsx'

In [16]:
zsdkap_dtypes = {
    'Warenempfänger': 'string',
    'Materialnummer': 'string',
    'Artikeltext': 'string',
    'Auftrag': 'string',
    'Kontroler MRP': 'string',
    'Menge': 'float',
    'WA-Datum': 'datetime64[ns]',
}

zsdkap_new_columns_names={
    'Warenempfänger': 'receiver',
    'Materialnummer': 'mat_number',
    'Artikeltext': 'mat_description',
    'Auftrag': 'customer_order_number',
    'Kontroler MRP': 'mrp_controller',
    'Menge': 'orders_quantity',
}

zsbe_dtypes = {
    'Materiał': 'string',
    'Zakład': 'string',
    'dowolne użycie': 'float',
    'zapas bezpieczeństwa': 'float',
}

zsbe_new_columns_names = {
    'Materiał': 'mat_number',
    'Zakład': 'plant',
    'dowolne użycie': 'stock_quantity',
    'Kontroler MRP': 'mrp_controller',
    'zapas bezpieczeństwa': 'safety_stock',
}

mb5t_dtypes = {
    'Materiał': 'string',
    'Zakład': 'string',
    'Ilość zamówienia': 'float',
    'Pozycja': 'string',
}

mb5t_new_columns_names = {
    'Materiał': 'mat_number',
    'Zakład': 'plant',
    'Zakład dostarczający': 'supplying_plant',
    'Ilość zamówienia': 'transit_quantity',
}

In [20]:
def get_nth_working_day(num_of_days: int) -> pd.Timestamp:
    """
    Returns the n-th working day (as pandas.Timestamp) 
    starting from the next working day.
    """
    current_date = date.today() + timedelta(days=1)  # start from tomorrow

    while num_of_days > 0:
        if current_date.weekday() < 5:  # Mon-Fri = 0-4
            num_of_days -= 1
            if num_of_days == 0:
                break
        current_date += timedelta(days=1)

    return pd.to_datetime(current_date)  # ensures pandas-compatible Timestamp


In [41]:
def get_zsdkap_df(mrp_controller, df, date_limit=None):
    tmp = df.copy()
    if date_limit is not None:
        tmp = tmp[tmp['WA-Datum'] <= date_limit]
    tmp = tmp[tmp['mrp_controller'] == mrp_controller]
    tmp = tmp[['mat_number', 'orders_quantity']]
    return tmp.groupby('mat_number', as_index=False).sum()

# 1. Load raw data once
raw_df = pd.read_excel(ZSDKAP_FILE_PATH, sheet_name='Sheet1', dtype=zsdkap_dtypes)
raw_df = raw_df.rename(columns=zsdkap_new_columns_names)

# 2. Base (total) dataframe
zsdkap_total_df = get_zsdkap_df('L1K', raw_df)

# 3. Horizons
horizons = [3, 5, 10]
dfs = []

for h in horizons:
    df_h = get_zsdkap_df('L1K', raw_df, date_limit=get_nth_working_day(h))
    df_h = df_h.rename(columns={'orders_quantity': f'orders_quantity_{h}_days'})
    dfs.append(df_h)

# 4. Merge everything
zsdkap_merged_df = zsdkap_total_df
for df_h in dfs:
    zsdkap_merged_df = zsdkap_merged_df.merge(df_h, on='mat_number', how='left')

sum_3_days = zsdkap_merged_df['orders_quantity_3_days'].sum()
sum_5_days = zsdkap_merged_df['orders_quantity_5_days'].sum()
sum_10_days = zsdkap_merged_df['orders_quantity_10_days'].sum()
sum_total = zsdkap_merged_df['orders_quantity'].sum()
print(f"3 days: {sum_3_days}, 5 days: {sum_5_days}, 10 days: {sum_10_days}, total: {sum_total}")

3 days: 360.0, 5 days: 448.0, 10 days: 496.0, total: 527.0


In [ ]:
zsbe_df = pd.read_excel(ZSBE_FILE_PATH, sheet_name='Sheet1', dtype=zsbe_dtypes)
zsbe_df = zsbe_df.rename(columns=zsbe_new_columns_names)
zsbe_df = zsbe_df[(zsbe_df['mrp_controller'] == 'L1K')]
zsbe_df = zsbe_df[['mat_number', 'stock_quantity', 'safety_stock']]
zsbe_df = zsbe_df.groupby('mat_number', as_index=False).sum()

In [6]:
zsbe_df

,mat_number,stock_quantity,safety_stock
0,2002530,0.0,0.0
1,2002531,0.0,0.0
2,2002532,0.0,0.0
3,2002533,0.0,0.0
4,2002534,0.0,0.0
...,...,...,...
1348,990510,0.0,0.0
1349,990511,0.0,0.0
1350,990843,0.0,0.0
1351,990844,0.0,0.0


In [ ]:
mb5t_df = pd.read_excel(MB5T_FROM_2101_TO_ALL_PLANTS_FILE_PATH, sheet_name='Sheet1', dtype=mb5t_dtypes)
mb5t_df = mb5t_df.rename(columns=mb5t_new_columns_names)
mb5t_df = mb5t_df[['mat_number', 'transit_quantity']]
mb5t_df = mb5t_df.groupby('mat_number', as_index=False).sum()
mb5t_df

,mat_number,transit_quantity
0,104558,90.0
1,2002106,2.0
2,2002530,1.0
3,2002538,6.0
4,2002541,1.0
...,...,...
255,990850,8.0
256,990934,3.0
257,991200,29.0
258,991440,5.0


In [8]:
zsdkap_zsbe_merged_df = pd.merge(zsdkap_df, zsbe_df, on='mat_number', how='outer')
zsdkap_zsbe_merged_df.fillna(0, inplace=True)
zsdkap_zsbe_merged_df

,mat_number,orders_quantity,stock_quantity,safety_stock
0,2002530,1.0,0.0,0.0
1,2002531,0.0,0.0,0.0
2,2002532,0.0,0.0,0.0
3,2002533,0.0,0.0,0.0
4,2002534,0.0,0.0,0.0
...,...,...,...,...
1348,990510,53.0,0.0,0.0
1349,990511,2.0,0.0,0.0
1350,990843,0.0,0.0,0.0
1351,990844,0.0,0.0,0.0


In [9]:
def calculate_to_be_produced_all(row):
    stock_quantity = row['stock_quantity'] + row['transit_quantity']
    if (stock_quantity - row['orders_quantity'] >= row['safety_stock']) and row['safety_stock'] > 0:
        return 0
    else:
        if row['orders_quantity'] + row['safety_stock'] - stock_quantity > 0:
            return row['orders_quantity'] + row['safety_stock'] - stock_quantity
        else:
            return 0
        
def calculate_to_be_produced_gr_c(row):
    stock_quantity = row['stock_quantity'] + row['transit_quantity']
    if stock_quantity < row['orders_quantity']:
        return row['orders_quantity'] - stock_quantity
    else:
        return 0

In [10]:
zsdkap_zsbe_mb5t_merged_df = pd.merge(zsdkap_zsbe_merged_df, mb5t_df, on='mat_number', how='left')
zsdkap_zsbe_mb5t_merged_df = zsdkap_zsbe_mb5t_merged_df.rename(columns=mb5t_new_columns_names)
zsdkap_zsbe_mb5t_merged_df.fillna(0, inplace=True)
zsdkap_zsbe_mb5t_merged_df['to_be_produced_all'] = zsdkap_zsbe_mb5t_merged_df.apply(calculate_to_be_produced_all, axis=1)
zsdkap_zsbe_mb5t_merged_df['to_be_produced_gr_c'] = zsdkap_zsbe_mb5t_merged_df.apply(calculate_to_be_produced_gr_c, axis=1)
zsdkap_zsbe_mb5t_merged_df

,mat_number,orders_quantity,stock_quantity,safety_stock,transit_quantity,to_be_produced_all,to_be_produced_gr_c
0,2002530,1.0,0.0,0.0,1.0,0.0,0.0
1,2002531,0.0,0.0,0.0,0.0,0.0,0.0
2,2002532,0.0,0.0,0.0,0.0,0.0,0.0
3,2002533,0.0,0.0,0.0,0.0,0.0,0.0
4,2002534,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1348,990510,53.0,0.0,0.0,0.0,53.0,53.0
1349,990511,2.0,0.0,0.0,0.0,2.0,2.0
1350,990843,0.0,0.0,0.0,0.0,0.0,0.0
1351,990844,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
zsdkap_zsbe_mb5t_merged_df.to_excel('excel_files/output/zsdkap_zsbe_mb5t_merged.xlsx', index=False)
print(zsdkap_zsbe_mb5t_merged_df['to_be_produced_all'].sum())
print(zsdkap_zsbe_mb5t_merged_df['to_be_produced_gr_c'].sum())

683.0
241.0


In [49]:
'sd' in ['ssd']

False

In [53]:
'sd' in ('ssd',)

False